In [1]:
from rl_agent import AgentGNN
import gym_zx
import gymnasium as gym
import torch
from torch_geometric.data import Batch, Data
import pyzx as zx

In [2]:
SrH_env = gym.make("zx-v0", qubits=12, depth=None, env_id=1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
rl_agent = AgentGNN(None,device=device).to(device)
rl_agent.load_state_dict(
        torch.load("/home/jnogue/qilimanjaro/Copt-cquere/rl-zx/state_dict_5x70_cquere_twoqubits.pt", map_location=torch.device("cpu"))
    )  
rl_agent.eval()


AgentGNN(
  (global_attention_critic): AttentionalAggregation(gate_nn=Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  ), nn=Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
  ))
  (critic_gnn): Sequential(
    (0): GATv2Conv(12, 32, heads=1)
    (1): ReLU()
    (2): GATv2Conv(32, 32, heads=1)
    (3): ReLU()
    (4): GATv2Conv(32, 32, heads=1)
    (5): ReLU()
    (6): GATv2Conv(32, 32, heads=1)
    (7): ReLU()
    (8): GATv2Conv(32, 32, heads=1)
    (9): ReLU()
  )
  (actor_gnn): Sequential(
    (0): GATv2Conv(17, 32, heads=1)
    (1): ReLU()
    (2): GATv2Conv(32, 32, heads=1)
    (3): ReLU()
    (4): GATv2Conv(32, 32, heads=1)
    (5): ReLU()
    (6): GATv2Conv(32, 32, heads=1)
    (7): ReLU()
   

In [3]:
obs0, reset_info = SrH_env.reset()
policy_items, value_items = reset_info["graph_obs"]
value_graph = [Data(x=value_items[0], edge_index=value_items[1], edge_attr=value_items[2])]
policy_graph = [Data(x=policy_items[0], edge_index=policy_items[1], edge_attr=policy_items[2], y=policy_items[3])]
next_obs_graph = (Batch.from_data_list(policy_graph), Batch.from_data_list(value_graph))

/home/jnogue/miniconda3/envs/cquere/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


In [4]:

done = False
while not done:
    
    action, _, _, value, logits, action_ids = rl_agent.get_action_and_value(next_obs_graph, device=device)
    next_obs, reward, done, deprecated, info = SrH_env.step(action_ids.cpu().numpy())
    policy_items, value_items = info["graph_obs"]
    value_graph = [Data(x=value_items[0], edge_index=value_items[1], edge_attr=value_items[2])]
    policy_graph = [Data(x=policy_items[0], edge_index=policy_items[1], edge_attr=policy_items[2], y=policy_items[3])]
    next_obs_graph = (Batch.from_data_list(policy_graph), Batch.from_data_list(value_graph))

/home/jnogue/miniconda3/envs/cquere/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


Win vs Pyzx:  -1  Episode Gates:  46 Cflow_gates:  33 Episode Len 146 Opt Episode Len 140


In [6]:
SrH_env.pyzx_data

/home/jnogue/miniconda3/envs/cquere/lib/python3.10/site-packages/gymnasium/core.py:297: UserWarning: WARN: env.pyzx_data to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.pyzx_data` for environment variables or `env.get_attr('pyzx_data')` that will search the reminding wrappers.
  logger.warn(


{'qubits': 12,
 'gates': 60,
 'tcount': 8,
 'clifford': 19,
 'CNOT': 26,
 'CZ': 7,
 'had': 5,
 'twoqubits': 33}

In [5]:
data = SrH_env.pyzx_data
print(data)
circuit = SrH_env.final_circuit
print(circuit.stats())



{'qubits': 12, 'gates': 60, 'tcount': 8, 'clifford': 19, 'CNOT': 26, 'CZ': 7, 'had': 5, 'twoqubits': 33}
Circuit  on 12 qubits with 66 gates.
        8 is the T-count
        58 Cliffords among which
        36 2-qubit gates (27 CNOT, 9 other) and
        2 Hadamard gates.


/home/jnogue/miniconda3/envs/cquere/lib/python3.10/site-packages/gymnasium/core.py:297: UserWarning: WARN: env.pyzx_data to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.pyzx_data` for environment variables or `env.get_attr('pyzx_data')` that will search the reminding wrappers.
  logger.warn(
/home/jnogue/miniconda3/envs/cquere/lib/python3.10/site-packages/gymnasium/core.py:297: UserWarning: WARN: env.final_circuit to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.final_circuit` for environment variables or `env.get_attr('final_circuit')` that will search the reminding wrappers.
  logger.warn(


In [83]:

print(circuit.stats())
print(circuit.to_qasm())


Circuit  on 12 qubits with 63 gates.
        8 is the T-count
        55 Cliffords among which
        36 2-qubit gates (35 CNOT, 1 other) and
        8 Hadamard gates.


In [9]:
import pyzx as zx
graph = SrH_env.final_circuit.to_graph()
print("Stats after RL-ZX\n", SrH_env.final_circuit.stats())
final_circuit = flow_opt(SrH_env.final_circuit)
print("Stats adding Calum as post processing\n", final_circuit.stats())


Stats after RL-ZX
 Circuit  on 12 qubits with 79 gates.
        8 is the T-count
        71 Cliffords among which
        46 2-qubit gates (40 CNOT, 6 other) and
        9 Hadamard gates.
Stats adding Calum as post processing
 Circuit  on 12 qubits with 84 gates.
        8 is the T-count
        76 Cliffords among which
        43 2-qubit gates (32 CNOT, 11 other) and
        9 Hadamard gates.


In [52]:
print(final_circuit.to_qasm())
print(final_circuit.stats())

OPENQASM 2.0;
include "qelib1.inc";
qreg q[12];
z q[1];
z q[2];
z q[3];
z q[4];
rz(1.5*pi) q[5];
z q[8];
z q[9];
rz(0.5*pi) q[10];
h q[10];
rz(0.5*pi) q[10];
rz(0.5*pi) q[11];
cz q[1], q[11];
cz q[9], q[11];
cz q[8], q[11];
cx q[11], q[7];
cx q[11], q[5];
cz q[4], q[11];
cz q[3], q[11];
cz q[2], q[11];
h q[11];
cx q[11], q[10];
rz(1.5*pi) q[11];
h q[11];
rz(1.9924411464549496*pi) q[11];
rz(-0.5*pi) q[7];
rz(1.9924411464549496*pi) q[10];
cx q[7], q[11];
rz(0.4924411464549496*pi) q[11];
cx q[5], q[11];
rz(0.9924411464549496*pi) q[11];
cx q[10], q[7];
cx q[5], q[10];
rz(0.9924411464549496*pi) q[10];
cx q[10], q[11];
rz(1.9924411464549496*pi) q[7];
cx q[5], q[7];
rz(0.9924411464549496*pi) q[7];
cx q[7], q[10];
cx q[11], q[7];
rz(0.9924411464549496*pi) q[7];
h q[7];
cz q[5], q[7];
cx q[7], q[10];
cz q[1], q[10];
cz q[9], q[10];
cz q[8], q[10];
cz q[4], q[10];
cz q[3], q[10];
cz q[2], q[10];
z q[10];
cx q[7], q[11];
cx q[10], q[5];
rz(1.5*pi) q[5];
x q[5];
x q[2];
x q[6];
x q[0];
x q[11];
cx

In [53]:
circuit_perm = SrH_env.circuit_up_to_perm
final_circuit_perm = flow_opt(circuit_perm)
print(final_circuit_perm.stats())


/home/jnogue/miniconda3/envs/cquere/lib/python3.10/site-packages/gymnasium/core.py:297: UserWarning: WARN: env.circuit_up_to_perm to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.circuit_up_to_perm` for environment variables or `env.get_attr('circuit_up_to_perm')` that will search the reminding wrappers.
  logger.warn(


Circuit  on 12 qubits with 64 gates.
        8 is the T-count
        56 Cliffords among which
        33 2-qubit gates (19 CNOT, 14 other) and
        5 Hadamard gates.


In [54]:
print(circuit_perm.to_qasm())

OPENQASM 2.0;
include "qelib1.inc";
qreg q[12];
swap q[10], q[11];
swap q[1], q[11];
h q[11];
h q[9];
h q[8];
h q[6];
h q[4];
h q[3];
h q[2];
h q[1];
h q[0];
h q[11];
h q[2];
h q[9];
cz q[5], q[11];
cz q[2], q[5];
cz q[1], q[9];
h q[7];
h q[10];
h q[5];
h q[4];
h q[8];
h q[3];
h q[1];
cz q[10], q[11];
cz q[8], q[11];
cz q[7], q[11];
cz q[5], q[11];
cz q[4], q[11];
cz q[3], q[11];
cz q[1], q[11];
rz(0.5*pi) q[11];
h q[11];
cz q[7], q[11];
h q[7];
cz q[10], q[11];
cz q[7], q[11];
rz(1.0075588535450504*pi) q[11];
h q[11];
cz q[7], q[11];
rz(1.0075588535450504*pi) q[7];
h q[7];
cx q[7], q[5];
rz(1.0*pi) q[10];
h q[10];
cz q[10], q[11];
rz(0.9924411464549496*pi) q[10];
h q[10];
cx q[10], q[5];
rz(1.0*pi) q[11];
h q[11];
rz(1.0075588535450504*pi) q[11];
h q[11];
cx q[11], q[5];
h q[7];
rz(1.0075588535450504*pi) q[7];
h q[7];
h q[10];
cz q[8], q[10];
cz q[7], q[10];
cz q[4], q[10];
cz q[3], q[10];
cz q[2], q[10];
rz(0.9924411464549496*pi) q[10];
h q[10];
h q[11];
cz q[10], q[11];
cz q[8], q[1

In [7]:
circ = zx.Circuit.from_qasm_file("/home/jnogue/qilimanjaro/Copt-cquere/rl-zx/cquere/circuits/after/12q-SrH-RL+cflow.qasm").to_basic_gates()
print(circ.depth())
c_before = zx.Circuit.from_qasm_file("/home/jnogue/qilimanjaro/Copt-cquere/rl-zx/cquere/circuits/before/SrH_2.1461_12q.qasm")
#zx.compare_tensors(circ,c_before)

37


In [70]:
circuit = zx.Circuit.from_qasm_file("/home/jnogue/qilimanjaro/Copt-cquere/rl-zx/cquere/circuits/before/SrH_2.1461_12q.qasm").to_basic_gates()
circ = flow_opt(circuit)
final_circ = full_reduce(circuit)
print("Stats cflow\n", circ.stats())
print("Stats full_reduce\n", final_circ.stats())


Stats cflow
 Circuit  on 12 qubits with 60 gates.
        8 is the T-count
        52 Cliffords among which
        33 2-qubit gates (26 CNOT, 7 other) and
        5 Hadamard gates.
Stats full_reduce
 Circuit  on 12 qubits with 57 gates.
        8 is the T-count
        49 Cliffords among which
        36 2-qubit gates (31 CNOT, 5 other) and
        4 Hadamard gates.


In [8]:
def basic_optimise(c):
    c1 = zx.basic_optimization(c.copy().split_phase_gates()).to_basic_gates()
    c2 = zx.basic_optimization(c.copy().split_phase_gates(), do_swaps=True).to_basic_gates()
    if c2.twoqubitcount() < c1.twoqubitcount(): return c2 # As this optimisation algorithm is targetted at reducting H-gates, we use the circuit with the smaller 2-qubit gate count here, either using SWAP rules or not.
    return c1
def flow_opt(c):

    c =zx.optimize.basic_optimization(zx.Circuit.from_graph(c.to_graph().copy()).split_phase_gates())
    g = c.to_graph()        
    zx.teleport_reduce(g)
    zx.to_graph_like(g)
    zx.flow_2Q_simp(g)
    c2 = zx.extract_simple(g).to_basic_gates()
    return basic_optimise(c2)

def flow_opt_graph(c):
    c = zx.optimize.basic_optimization(zx.Circuit.from_graph(c.to_graph().copy()).split_phase_gates())
    g = c.to_graph()
    basic_circ = zx.optimize.basic_optimization(zx.Circuit.from_graph(g.copy()).split_phase_gates())
    graph = basic_circ.to_graph()
    zx.simplify.to_gh(graph)
    zx.simplify.spider_simp(graph, quiet=True)
    zx.to_graph_like(graph)
    zx.flow_2Q_simp(graph)
    circuit = zx.extract_simple(graph).to_basic_gates()
    return basic_optimise(circuit)

def full_reduce(c):
    c = zx.optimize.basic_optimization(zx.Circuit.from_graph(c.to_graph().copy()).split_phase_gates())
    g = c.to_graph()        
    zx.teleport_reduce(g)
    zx.to_graph_like(g)
    zx.full_reduce(g)
    circ = zx.extract_circuit(g).to_basic_gates()
    return basic_optimise(circ)
